## 🧠 Project Objectives: Agno Agent with Email Flow

This agent is designed to serve as a conversational assistant inside a Jupyter notebook using [Agno](https://docs.agno.com/). It supports Q&A and a structured email-sending workflow using GROQ and the `@tool` decorator system.

### 🎯 Core Objectives

1. **Conversational AI Agent**  
   Create an agent that interacts with users, answers questions, and handles requests in natural language.

2. **Email Workflow Integration**  
   When a user asks to send an email to a customer:
   - Find the customer’s email from a mock database using a tool (`find_customer_email`).
   - Draft the email based on the user's input.

3. **Feedback & Review Loop**  
   - Show the drafted email to the user for review.
   - If the user responds with "refine" or similar feedback, regenerate the email.
   - If the user says "ok" or "send", proceed to send the email.

4. **Tool-Driven Architecture**  
   - Implement tools using `@tool` for modular logic:
     - `find_customer_email`: Lookup customer emails.
     - `send_email`: Simulate sending email messages.

5. **Use of Environment Variables**  
   - Load secrets such as API keys securely using `python-dotenv`.

6. **GROQ + OpenAI Backend**  
   - Use GROQ or OpenAI as the model backend to power the agent’s reasoning and response generation.

7. **Notebook-First Development**  
   - Keep everything modular and testable within a Jupyter notebook environment.

---


In [36]:
# ! pip -q install openai agno
# ! pip install pandas
# ! pip install python-dotenv
# ! pip install duckduckgo-search
# ! pip install -U tavily-python



# Load Secrets

In [37]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path='../backend/.env')  # Adjust if not running from 'notebooks/'


# Access the variables
openai_key = os.getenv("OPENAI_API_KEY")
gmail_user = os.getenv("GMAIL_ADDRESS")
gmail_pass = os.getenv("GMAIL_APP_PASSWORD")
tavily_key = os.getenv("TAVILY_API_KEY")



print("Loaded API Key:", bool(openai_key))  # Should print True if loaded


Loaded API Key: True


In [38]:
import os
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools import tool
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.tavily import TavilyTools


## Create Mock Customer Dataset

In [39]:
import pandas as pd

# Create a sample user database
user_df = pd.DataFrame([
    {"username": "nayeem", "email": "nayeem60151126@gmail.com", "name": "Nayeem"},
    {"username": "alice", "email": "alice.j@example.com", "name": "Alice Johnson"},
    {"username": "bob", "email": "bob.smith@example.com", "name": "Bob Smith"},
])

user_df

,username,email,name
0,nayeem,nayeem60151126@gmail.com,Nayeem
1,alice,alice.j@example.com,Alice Johnson
2,bob,bob.smith@example.com,Bob Smith


## Define Tools Using Agno

In [40]:
from agno.tools import tool

def find_customer_email(name: str) -> str:
    """
    Retrieve the email address of a customer by name from user_df.
    """
    match = user_df[user_df["name"].str.lower() == name.lower()]
    if not match.empty:
        return match.iloc[0]["email"]
    return "Email not found."


In [41]:
from agno.tools import tool

def find_customer_email(name: str) -> str:
    """
    Retrieve the email address of a customer by name from user_df.
    """
    match = user_df[user_df["name"].str.lower() == name.lower()]
    if not match.empty:
        return match.iloc[0]["email"]
    return "Email not found."

import smtplib
from email.mime.text import MIMEText
from agno.tools import tool

def send_email(to: str, subject: str, body: str) -> str:
    """
    Send an email using Gmail's SMTP server.
    """
    gmail_address = gmail_user
    gmail_password = gmail_pass

    if not gmail_address or not gmail_password:
        return "SMTP credentials are not set properly in the environment."

    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = gmail_address
    msg["To"] = to

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(gmail_address, gmail_password)
            server.send_message(msg)
        return "Email sent successfully."
    except Exception as e:
        return f"Failed to send email: {e}"


#  Run the Agent

In [42]:
from typing import Dict, Any, Callable
from agno.agent import Agent
from agno.tools import Toolkit, tool
from agno.exceptions import StopAgentRun
from rich.console import Console
from rich.prompt import Prompt

console = Console()

# Create the agent
agent = Agent(
    model=OpenAIChat(id="gpt-4o", api_key=openai_key),
    tools=[find_customer_email, send_email,TavilyTools()],
    instructions=[
        "You are an email assistant that can:",
        "1. Look up customer email addresses",
        "2. Send emails to specified addresses",
        "3. When asked to send an email:",
        "1. Draft the email professionally and show it to the user",
        "2. Ask for feedback and make any requested changes",
        "3. Only send after explicit confirmation",
        "4. Be helpful and maintain a professional tone",
        "5. Use Tavily Tool for web search if user needs something from the internet",
        "Use the appropriate tool based on the user's request."
    ],
    # Add memory configuration
    add_history_to_messages=True,  # Enable chat history
    num_history_responses=3,       # Keep last 3 exchanges in memory
    show_tool_calls=True,
    markdown=True
)


In [43]:
def run_agent_loop():
    console.print("[bold green]Email Assistant is running. Type 'exit' to quit.[/bold green]")
    while True:
        user_input = Prompt.ask("\n[bold blue]You[/bold blue]").strip()
        if user_input.lower() in ["exit", "quit"]:
            console.print("[bold yellow]Goodbye![/bold yellow]")
            break
        try:
            agent.print_response(
                user_input,
                stream=True,
                console=console
            )
        except Exception as e:
            console.print(f"[red]Error:[/red] {e}")

# Start the loop
# run_agent_loop()


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import traceback
import re

# --- Helper: Sanitize malformed markdown links ---
def clean_malformed_links(text):
    text = re.sub(r'\[([^\]]*)\]\(\s*\)', r'\1', text)
    text = re.sub(r'\[\]\(([^)]*)\)', r'\1', text)
    return text

# --- UI Elements ---

output_area = widgets.Output(layout={
    'border': '1px solid lightgray',
    'height': '300px',
    'overflow_y': 'auto',
    'padding': '10px',
    'background_color': '#f9f9f9'
})

input_box = widgets.Text(
    placeholder='Type your message here...',
    layout=widgets.Layout(flex='1 1 auto', width='auto')
)

send_button = widgets.Button(description="Send", button_style='success', tooltip="Send message")
exit_button = widgets.Button(description="Exit", button_style='danger', tooltip="Exit chat")

input_controls = widgets.HBox([input_box, send_button, exit_button])
chat_ui = widgets.VBox([output_area, input_controls])

display(HTML(
    "<h4 style='color:green;'>📧 Email Assistant is running</h4>"
    "<small>Type your message and press 'Send' or 'Enter'. Type 'exit' or press 'Exit' to quit.</small>"
))
display(chat_ui)

# --- Core Logic ---

def exit_chat():
    input_box.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("\nAssistant: Goodbye! 👋")

def handle_message(user_input):
    """Process user input and handle agent responses safely."""
    with output_area:
        print(f"\nYou: {user_input}")
    
    if user_input.lower() in ['exit', 'quit']:
        exit_chat()
        return

    try:
        with output_area:
            agent.print_response(
                user_input,
                stream=True,
                console=console
            )
    except IndexError as e:
        with output_area:
            print("⚠️ Markdown rendering failed. Retrying without streaming or markdown...")
        try:
            with output_area:
                agent.print_response(
                    user_input,
                    stream=False,
                    markdown=False,
                    console=console
                )
        except Exception as fallback_error:
            with output_area:
                print("⚠️ Fallback attempt also failed.")
                print(f"Details: {fallback_error}")
                traceback.print_exc()
    except Exception as e:
        with output_area:
            print("⚠️ An unexpected error occurred.")
            print(f"Details: {e}")
            traceback.print_exc()


def on_send_clicked(_):
    user_input = input_box.value.strip()
    input_box.value = ""
    if user_input:
        handle_message(user_input)

def on_enter_pressed(change):
    if change['name'] == 'value' and change['new'] == "":
        on_send_clicked(None)

# --- Event Bindings ---
send_button.on_click(on_send_clicked)
exit_button.on_click(lambda _: exit_chat())
input_box.observe(on_enter_pressed, names='value')
